In [45]:
import pandas as pd

In [46]:
df = pd.read_csv('datasets/focus.csv')
df

,model,year,price,transmission,mileage,fuelType,engineSize
0,Focus,2016,8000,Manual,38852,Petrol,1.0
1,Focus,2019,13400,Manual,11952,Petrol,1.0
2,Focus,2019,14600,Manual,22142,Petrol,1.5
3,Focus,2016,9450,Manual,14549,Diesel,1.6
4,Focus,2015,9999,Manual,7010,Diesel,1.6
...,...,...,...,...,...,...,...
5449,Focus,2019,18745,Manual,7855,Diesel,2.0
5450,Focus,2019,16350,Manual,13891,Petrol,1.0
5451,Focus,2019,16850,Manual,13452,Petrol,1.0
5452,Focus,2019,17310,Automatic,13376,Petrol,1.0


In [47]:
df['model'] = df['model'].str.strip()

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5454 entries, 0 to 5453
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         5454 non-null   object 
 1   year          5454 non-null   int64  
 2   price         5454 non-null   int64  
 3   transmission  5454 non-null   object 
 4   mileage       5454 non-null   int64  
 5   fuelType      5454 non-null   object 
 6   engineSize    5454 non-null   float64
dtypes: float64(1), int64(3), object(3)
memory usage: 298.4+ KB


In [49]:
df.isnull().sum()

model           0
year            0
price           0
transmission    0
mileage         0
fuelType        0
engineSize      0
dtype: int64

## Importing ML Libraries

In [50]:
import numpy as np
import sklearn.metrics as m
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
# cross validation
from sklearn.model_selection import cross_val_score
# grid search
from sklearn.model_selection import GridSearchCV
# outlier
from sklearn.neighbors import LocalOutlierFactor
from sklearn.compose import ColumnTransformer

## Feature Selection

In [51]:
X = df.drop(columns=['price'])
y = df['price']

In [52]:
X# clean data

,model,year,transmission,mileage,fuelType,engineSize
0,Focus,2016,Manual,38852,Petrol,1.0
1,Focus,2019,Manual,11952,Petrol,1.0
2,Focus,2019,Manual,22142,Petrol,1.5
3,Focus,2016,Manual,14549,Diesel,1.6
4,Focus,2015,Manual,7010,Diesel,1.6
...,...,...,...,...,...,...
5449,Focus,2019,Manual,7855,Diesel,2.0
5450,Focus,2019,Manual,13891,Petrol,1.0
5451,Focus,2019,Manual,13452,Petrol,1.0
5452,Focus,2019,Automatic,13376,Petrol,1.0


## Column Extraction

In [53]:
bin_cols = ['transmission', 'fuelType']
num_cols = ['year', 'mileage', 'engineSize']
cat_cols = ['model']

## Preprocessing Pipeline

In [54]:
# create pipeline
num_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
])
binary_pipeline = Pipeline([
    ('one_hot_encoder', OrdinalEncoder())
])
cat_pipeline = Pipeline([
    ('one_hot_encoder', OneHotEncoder(drop='first'))
])

# create column transformer
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('binary', binary_pipeline, bin_cols),
    ('cat', cat_pipeline, cat_cols)
])

preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('std_scaler',
                                                  StandardScaler())]),
                                 ['year', 'mileage', 'engineSize']),
                                ('binary',
                                 Pipeline(steps=[('one_hot_encoder',
                                                  OrdinalEncoder())]),
                                 ['transmission', 'fuelType']),
                                ('cat',
                                 Pipeline(steps=[('one_hot_encoder',
                                                  OneHotEncoder(drop='first'))]),
                                 ['model'])])

## Outlier Removal
- this help to remove the outliers from the dataset

In [55]:
outlier_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('outlier', LocalOutlierFactor(n_neighbors=20, contamination=0.1))
])

outlier_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('std_scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'mileage',
                                                   'engineSize']),
                                                 ('binary',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OrdinalEncoder())]),
                                                  ['transmission', 'fuelType']),
                                                 ('cat',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['model'])])),
                ('outlier', LocalOutlierFactor(contamination=0.1))])

In [56]:
yhat = outlier_pipeline.fit_predict(X)
X = X[yhat==1].copy()
y = y[yhat==1].copy()

In [57]:
df = pd.concat([X, y], axis=1)

In [58]:
X# cleane data

,model,year,transmission,mileage,fuelType,engineSize
0,Focus,2016,Manual,38852,Petrol,1.0
1,Focus,2019,Manual,11952,Petrol,1.0
5,Focus,2016,Automatic,26796,Petrol,1.0
6,Focus,2018,Manual,16128,Petrol,1.0
8,Focus,2018,Manual,30953,Petrol,1.5
...,...,...,...,...,...,...
5449,Focus,2019,Manual,7855,Diesel,2.0
5450,Focus,2019,Manual,13891,Petrol,1.0
5451,Focus,2019,Manual,13452,Petrol,1.0
5452,Focus,2019,Automatic,13376,Petrol,1.0


## Model Pipeline

In [59]:
pipeline1 = Pipeline([
    ('preprocessor', preprocessor),
    ('model', DecisionTreeRegressor())
])
pipeline1

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('std_scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'mileage',
                                                   'engineSize']),
                                                 ('binary',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OrdinalEncoder())]),
                                                  ['transmission', 'fuelType']),
                                                 ('cat',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['model'])])),
                ('model', DecisionTreeRegressor())])

In [60]:
pipeline2 = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor())
])
pipeline2

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('std_scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'mileage',
                                                   'engineSize']),
                                                 ('binary',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OrdinalEncoder())]),
                                                  ['transmission', 'fuelType']),
                                                 ('cat',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['model'])])),
                ('model', RandomForestRegressor())])

In [61]:
pipeline3 = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])
pipeline3

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('std_scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'mileage',
                                                   'engineSize']),
                                                 ('binary',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OrdinalEncoder())]),
                                                  ['transmission', 'fuelType']),
                                                 ('cat',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['model'])])),
                ('model', LinearRegression())])

In [62]:
pipeline4 = Pipeline([
    ('preprocessor', preprocessor),
    ('model', SVR())
])
pipeline4

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('std_scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'mileage',
                                                   'engineSize']),
                                                 ('binary',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OrdinalEncoder())]),
                                                  ['transmission', 'fuelType']),
                                                 ('cat',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['model'])])),
                ('model', SVR())])

## Training Model and Evaluation

In [63]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)
model1 = pipeline1.fit(train_X, train_y)
yhat = model1.predict(test_X)
mae = m.mean_absolute_error(test_y, yhat)
mse = m.mean_squared_error(test_y, yhat)
r2 = m.r2_score(test_y, yhat)
rmse = np.sqrt(mse)
print("Decision Tree Regressor")
print(f'MAE: {mae} - tells us how far off our predictions are on average.')
print(f'MSE: {mse} - tells us how far off our predictions are on average squared.')
print(f'RMSE: {rmse} - tells us how far off our predictions are on average squared root.')
print(f'R2: {r2} - tells us how much of the variance in the target variable is explained by the model.')

Decision Tree Regressor
MAE: 1032.8071840752593 - tells us how far off our predictions are on average.
MSE: 2433088.210754609 - tells us how far off our predictions are on average squared.
RMSE: 1559.8359563603506 - tells us how far off our predictions are on average squared root.
R2: 0.8778225571328352 - tells us how much of the variance in the target variable is explained by the model.


In [64]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)
model2 = pipeline2.fit(train_X, train_y)
yhat = model2.predict(test_X)
mae = m.mean_absolute_error(test_y, yhat)
mse = m.mean_squared_error(test_y, yhat)
r2 = m.r2_score(test_y, yhat)
rmse = np.sqrt(mse)
print("Random Forest Regressor")
print(f'MAE: {mae} - tells us how far off our predictions are on average.')
print(f'MSE: {mse} - tells us how far off our predictions are on average squared.')
print(f'RMSE: {rmse} - tells us how far off our predictions are on average squared root.')
print(f'R2: {r2} - tells us how much of the variance in the target variable is explained by the model.')

Random Forest Regressor
MAE: 899.1928287641258 - tells us how far off our predictions are on average.
MSE: 1850458.8905763554 - tells us how far off our predictions are on average squared.
RMSE: 1360.3157319447407 - tells us how far off our predictions are on average squared root.
R2: 0.9070792688969912 - tells us how much of the variance in the target variable is explained by the model.


In [65]:
model3 = pipeline3.fit(train_X, train_y)
yhat = model3.predict(test_X)
mae = m.mean_absolute_error(test_y, yhat)
mse = m.mean_squared_error(test_y, yhat)
r2 = m.r2_score(test_y, yhat)
rmse = np.sqrt(mse)
print("Linear Regression")
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R2: {r2}')

Linear Regression
MAE: 1300.1906131339276
MSE: 3025553.52903842
RMSE: 1739.411834223977
R2: 0.8480719310538297


In [66]:
model4 = pipeline4.fit(train_X, train_y)
yhat = model4.predict(test_X)
mae = m.mean_absolute_error(test_y, yhat)
mse = m.mean_squared_error(test_y, yhat)
r2 = m.r2_score(test_y, yhat)
rmse = np.sqrt(mse)
print("Support Vector Regression")
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R2: {r2}')

Support Vector Regression
MAE: 3396.894957558969
MSE: 17813137.982210036
RMSE: 4220.561334966006
R2: 0.10551387389633426


In [67]:
df['dtree_pred'] = model1.predict(df.drop(columns=['price']))
df['rf_pred'] = model2.predict(df.drop(columns=['price']))
df['lr_pred'] = model3.predict(df.drop(columns=['price']))
df['svr_pred'] = model4.predict(df.drop(columns=['price']))

## Visualization

In [68]:
# histogram
fig = px.histogram(df, x='price', nbins=100, title='Price Distribution')
fig.show()

## Saving the model

In [69]:
import joblib

In [70]:
joblib.dump(model1, 'focus/dtree_model.pkl')
joblib.dump(model2, 'focus/rf_model.pkl')
joblib.dump(model3, 'focus/lr_model.pkl')
joblib.dump(model4, 'focus/svr_model.pkl')

['focus/svr_model.pkl']

In [71]:
X.fuelType.unique()

array(['Petrol', 'Diesel'], dtype=object)

In [72]:
X.model.unique()

array(['Focus'], dtype=object)

In [73]:
X.transmission.unique()

array(['Manual', 'Automatic', 'Semi-Auto'], dtype=object)

In [74]:
X

,model,year,transmission,mileage,fuelType,engineSize
0,Focus,2016,Manual,38852,Petrol,1.0
1,Focus,2019,Manual,11952,Petrol,1.0
5,Focus,2016,Automatic,26796,Petrol,1.0
6,Focus,2018,Manual,16128,Petrol,1.0
8,Focus,2018,Manual,30953,Petrol,1.5
...,...,...,...,...,...,...
5449,Focus,2019,Manual,7855,Diesel,2.0
5450,Focus,2019,Manual,13891,Petrol,1.0
5451,Focus,2019,Manual,13452,Petrol,1.0
5452,Focus,2019,Automatic,13376,Petrol,1.0
